# EM4QS Demo 22.01.2026


## Library installation
We have not yet publicised the library in Pypi, so for now you can build it from source. 

```
git clone https://github.com/QuantumHel/syn.git
cd syn
cargo build
pip install .
```

In [1]:
from qiskit.quantum_info import Clifford, Operator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import PermutationGate

from synpy.qiskit.plugin import SynPyCliffordPlugin, qiskit_to_synir
from synpy.synpy_rust import QiskitSynIR

circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.rz(1.5, 1)
sample_circuit = circuit.copy()

print(circuit)

     ┌───┐                
q_0: ┤ H ├──■─────────────
     └───┘┌─┴─┐┌─────────┐
q_1: ─────┤ X ├┤ Rz(1.5) ├
          └───┘└─────────┘
q_2: ─────────────────────
                          


In [2]:
def our_transpile(circuit: QuantumCircuit) -> QuantumCircuit:
    synir_result = QiskitSynIR(circuit.copy_empty_like())
    pe_wrap = qiskit_to_synir(circuit)
    pe_wrap.synthesize_to_qiskit(synir_result)
    perm1 = synir_result.get_permutation()
    perm2 = [perm1.index(i) for i in range(len(perm1))]
    print(perm2)
    new_circuit = synir_result.get_circuit()
    new_circuit.append(PermutationGate(perm2), new_circuit.qubits, [])
    return synir_result.get_circuit()

In [3]:
new_circuit = our_transpile(circuit)
print(new_circuit)

[0, 1, 2]
     ┌───┐                ┌──────────────┐
q_0: ┤ H ├──■─────────────┤0             ├
     └───┘┌─┴─┐┌─────────┐│              │
q_1: ─────┤ X ├┤ Rz(1.5) ├┤1 Permutation ├
          └───┘└─────────┘│              │
q_2: ─────────────────────┤2             ├
                          └──────────────┘


In [4]:
def check_equivalent(circuit1: QuantumCircuit, circuit2: QuantumCircuit) -> bool:
    op1 = Operator.from_circuit(circuit1)
    op2 = Operator.from_circuit(circuit2)
    b = op1.equiv(op2)
    assert b
    return b

In [5]:
def stats(circuit):
    print(circuit.count_ops())
    print(circuit.draw(fold=200))

In [6]:
check_equivalent(new_circuit, sample_circuit)

True

# Single qubit

In [7]:
sqc = QuantumCircuit(1)
sqc.rz(0.234, 0)
stats(sqc)

OrderedDict({'rz': 1})
   ┌───────────┐
q: ┤ Rz(0.234) ├
   └───────────┘


In [8]:
qiskit_sqc = transpile(sqc, optimization_level=3, basis_gates=["cx", "h", "rz"])
stats(qiskit_sqc)

OrderedDict({'rz': 1})
   ┌───────────┐
q: ┤ Rz(0.234) ├
   └───────────┘


In [9]:
syn_sqc = our_transpile(sqc)
stats(syn_sqc)

[0]
OrderedDict({'rz': 1, 'permutation': 1})
   ┌───────────┐┌─────────────┐
q: ┤ Rz(0.234) ├┤ Permutation ├
   └───────────┘└─────────────┘


# Simple 2 qubits

In [10]:
tqc = QuantumCircuit(2)
tqc.rz(0.423, 0)
tqc.rz(0.423, 0)
tqc.cx(0,1)
tqc.rz(0.145,1)
tqc.rz(0.145,1)
stats(tqc)

OrderedDict({'rz': 4, 'cx': 1})
     ┌───────────┐┌───────────┐                               
q_0: ┤ Rz(0.423) ├┤ Rz(0.423) ├──■────────────────────────────
     └───────────┘└───────────┘┌─┴─┐┌───────────┐┌───────────┐
q_1: ──────────────────────────┤ X ├┤ Rz(0.145) ├┤ Rz(0.145) ├
                               └───┘└───────────┘└───────────┘


In [11]:
qiskit_tqc = transpile(tqc, optimization_level=3, basis_gates=["cx", "h", "rz"])
stats(tqc)

OrderedDict({'rz': 4, 'cx': 1})
     ┌───────────┐┌───────────┐                               
q_0: ┤ Rz(0.423) ├┤ Rz(0.423) ├──■────────────────────────────
     └───────────┘└───────────┘┌─┴─┐┌───────────┐┌───────────┐
q_1: ──────────────────────────┤ X ├┤ Rz(0.145) ├┤ Rz(0.145) ├
                               └───┘└───────────┘└───────────┘


In [12]:
syn_tqc = our_transpile(tqc)
stats(syn_tqc)

[0, 1]
OrderedDict({'rz': 2, 'cx': 1, 'permutation': 1})
     ┌───────────┐                 ┌──────────────┐
q_0: ┤ Rz(0.846) ├──■──────────────┤0             ├
     └───────────┘┌─┴─┐┌──────────┐│  Permutation │
q_1: ─────────────┤ X ├┤ Rz(0.29) ├┤1             ├
                  └───┘└──────────┘└──────────────┘


# Random circuits

In [13]:
from qiskit.circuit.library import QuantumVolume

qv = QuantumVolume(5).decompose(reps=2)
print(qv.draw(fold=200))
qv.count_ops()

global phase: 2.5995
      ┌──────────────────────────┐                                                                                                                                                                »
q_0: ─┤ U(3.0318,1.5958,0.22901) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     ┌┴──────────────────────────┤┌───┐┌───────────────────────────┐                              ┌───┐┌───────────────────────────┐                             ┌───┐┌──────────────────────────┐»
q_1: ┤ U(1.0913,1.9493,-0.49529) ├┤ X ├┤ U(1.5472,-3.0433,0.23436) ├──────────────────────────────┤ X ├┤ U(1.6714,0.01024,-1.4692) ├─────────────────────────────┤ X ├┤ U(1.0659,2.2338,-2.6769) ├»
     ├───────────────────────────┤└─┬─┘└───────────────────────────┘    ┌────────────────────┐    └─┬─┘└───────────────────────────┘    ┌───────────────────┐    └─┬─┘└──────────────────────────┘»

OrderedDict([('u', 80), ('cx', 30)])

In [14]:
qiskit_qv = transpile(qv, optimization_level=3, basis_gates=["cx", "h", "rz"])
print(qiskit_qv.draw(fold=200))
qiskit_qv.count_ops()

global phase: 4.9165
     ┌─────────────┐ ┌──────────┐┌───┐┌──────────┐┌────────────┐┌──────────┐┌───┐┌──────────┐┌────────────┐                                                                                    »
q_0: ┤ Rz(-2.6315) ├─┤ Rz(-π/2) ├┤ H ├┤ Rz(-π/2) ├┤ Rz(3.1507) ├┤ Rz(-π/2) ├┤ H ├┤ Rz(-π/2) ├┤ Rz(7.5481) ├────────────────────────────────────────────────────────────────────────────────────»
     ├─────────────┴┐├──────────┤├───┤├──────────┤├────────────┤├──────────┤├───┤├──────────┤├────────────┤     ┌──────────┐  ┌──────────┐    ┌───┐    ┌──────────┐┌────────────┐ ┌──────────┐ »
q_1: ┤ Rz(-0.45503) ├┤ Rz(-π/2) ├┤ H ├┤ Rz(-π/2) ├┤ Rz(4.3271) ├┤ Rz(-π/2) ├┤ H ├┤ Rz(-π/2) ├┤ Rz(8.2156) ├──■──┤ Rz(-π/2) ├──┤ Rz(-π/2) ├────┤ H ├────┤ Rz(-π/2) ├┤ Rz(3.7172) ├─┤ Rz(-π/2) ├─»
     └┬────────────┬┘├──────────┤├───┤├──────────┤├────────────┤├──────────┤├───┤├──────────┤├────────────┤  │  └──────────┘  ├──────────┤ ┌──┴───┴───┐└──┬───┬───┘└┬──────────┬┘┌┴──────────┴┐»
q_2: ─┤ Rz(2.9

OrderedDict([('rz', 462), ('h', 132), ('cx', 27)])

In [15]:
syn_qv = our_transpile(qv)
print(syn_qv.draw(fold=200))
syn_qv.count_ops()

[3, 2, 0, 4, 1]
global phase: 2.5995
     ┌─────────────┐┌───┐ ┌────────────┐┌───┐┌────────────┐┌───┐                                                                                                                          »
q_0: ┤ Rz(-1.0607) ├┤ H ├─┤ Rz(3.1325) ├┤ H ├┤ Rz(6.5891) ├┤ H ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     ├─────────────┤├───┤ ├────────────┤├───┤├────────────┤├───┤     ┌───┐                                   ┌─────────────┐┌───┐┌────────────┐┌───┐┌────────────┐    ┌───┐          ┌───┐»
q_1: ┤ Rz(-2.0258) ├┤ H ├─┤ Rz(1.1856) ├┤ H ├┤ Rz(3.5032) ├┤ H ├──■──┤ H ├────────────────────────────────■──┤ Rz(-1.3364) ├┤ H ├┤ Rz(1.5472) ├┤ H ├┤ Rz(4.8107) ├────┤ H ├───────■──┤ H ├»
     └┬───────────┬┘├───┤ ├────────────┤├───┤├────────────┤├───┤  │  └───┘     ┌───┐     ┌─────────────┐  │  └────┬───┬────┘└───┘└───┬───┬────┘└───┘└────────────┘    ├───┤       │  └───┘»
q_2: ─┤ Rz(1.384) ├─┤ H

OrderedDict([('h', 188), ('rz', 167), ('cx', 41), ('permutation', 1)])

In [16]:
check_equivalent(qiskit_qv, qv)

True

In [17]:
check_equivalent(qv, syn_qv)

True

# Compiling Shor's 21
The [quantum program to factor 21](https://algassert.com/post/2500) is much too long to execute on current hardware. But we can improve the cost with our library.

In [18]:
from qiskit import qasm2

original_shor = qasm2.load("factor21.qasm")
print(original_shor.count_ops())
original_shor.draw(fold=200)

/var/folders/3q/vjbn2sl16r1g41lg5pk752khfzf5fm/T/ipykernel_96891/1544880673.py:3: FutureWarning: Conditioned gates in qasm2 will be loaded as an IfElseOp starting in Qiskit 2.0
  original_shor = qasm2.load("factor21.qasm")


OrderedDict({'ccx': 369, 'cx': 191, 'x': 73, 'u': 45, 'h': 20, 'measure': 10})


┌───┐┌───┐     ┌───┐                         ┌───┐          ┌───┐                                             ┌───┐                                   ┌───┐     ┌───┐          ┌───┐     »
    val0: ┤ X ├┤ X ├─────┤ X ├─────────────────────────┤ X ├──■───────┤ X ├──■────────────────────────■─────────────────┤ X ├──────────────────────■────────────┤ X ├──■──┤ X ├───────■──┤ X ├─────»
          └───┘└─┬─┘┌───┐└─┬─┘                         └─┬─┘  │       └─┬─┘  │            ┌───┐       │                 └─┬─┘┌───┐                 │            └─┬─┘  │  └─┬─┘       │  └─┬─┘     »
    val1: ───────┼──┤ X ├──┼─────────────────────────────┼────┼─────────┼────┼────────────┤ X ├───────┼─────────■─────────┼──┤ X ├───────■─────────┼─────────■────┼────┼────■─────────┼────┼───────»
          ┌───┐  │  └─┬─┘  │  ┌───┐               ┌───┐  │  ┌─┴─┐       │    │            └─┬─┘       │  ┌───┐  │  ┌───┐  │  └─┬─┘     ┌─┴─┐       │         │    │  ┌─┴─┐  │         │    │       »
    val2: ┤ X ├──┼────┼────┼──┤ X ├───────■───────┤ X ├──■──┤ X ├───────■────■──────────────┼─────────■──┤ X ├──┼──┤ X ├──┼────┼────■──┤ X ├──■────┼────■────┼────┼──┤ X ├──┼─────────■────┼───────»
          └─┬─┘  │    │    │  └─┬─┘       │       └─┬─┘  │  └─┬─┘┌───┐  │    │  ┌───┐       │         │  └─┬─┘┌─┴─┐└───┘  │    │    │  └─┬─┘┌─┴─┐  │  ┌─┴─┐┌─┴─┐  │  └─┬─┘  │         │    │       »
    val3: ──■────┼────┼────┼────┼────■────┼────■────┼────■────┼──┤ X ├──■────┼──┤ X ├───────■─────────┼────┼──┤ X ├──■────■────■────■────┼──┤ X ├──■──┤ X ├┤ X ├──■────■────┼────■────┼────■────■──»
                 │    │    │    │    │  ┌─┴─┐  │    │         │  └─┬─┘     ┌─┴─┐└─┬─┘┌───┐  │  ┌───┐┌─┴─┐  │  └─┬─┘┌─┴─┐  │    │  ┌─┴─┐  │  └───┘┌─┴─┐└───┘└─┬─┘  │         │  ┌─┴─┐┌─┴─┐       │  »
    val4: ───────┼────┼────┼────■────┼──┤ X ├──┼────■─────────┼────┼───────┤ X ├──┼──┤ X ├──■──┤ X ├┤ X ├──┼────┼──┤ X ├──┼────■──┤ X ├──┼───────┤ X ├───────┼────┼─────────┼──┤ X ├┤ X ├───────┼──»
          ┌───┐  │    │    │    │    │  └─┬─┘  │    │         │    │       └───┘  │  └───┘     └───┘└───┘  │    │  └───┘  │       └───┘  │       └───┘       │    │         │  └───┘└───┘       │  »
    exp0: ┤ H ├──■────■────┼────┼────┼────┼────┼────┼─────────┼────┼──────────────┼────────────────────────┼────┼─────────┼──────────────┼───────────────────┼────┼─────────┼───────────────────┼──»
          ├───┤            │    │  ┌─┴─┐  │  ┌─┴─┐  │         │    │              │                        │    │         │              │                   │    │         │                 ┌─┴─┐»
    exp1: ┤ H ├────────────■────■──┤ X ├──■──┤ X ├──■─────────■────■──────────────■────────────────────────■────■─────────■──────────────■───────────────────■────■─────────■─────────────────┤ X ├»
          ├───┤                    └───┘     └───┘                                                                                                                                            └───┘»
    exp2: ┤ H ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
          ├───┤                                                                                                                                                                                    »
    exp3: ┤ H ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
          ├───┤                                                                                                                                                                                    »
    exp4: ┤ H ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
          ├───┤                                                       

This circuit contains gates that are not yet natively supported by our library. 
The QFT is implemented as a fourier basis umeasurement, which we can temporarily remove and place back.

In [19]:
no_qft = original_shor.copy_empty_like()
for gate in original_shor.data: 
    if gate.name not in ["measure", "u"]:
        no_qft.compose(gate, gate.qubits, gate.clbits, inplace=True)
no_qft.count_ops()

OrderedDict([('ccx', 369), ('cx', 191), ('x', 73), ('h', 20)])

In [20]:
from qiskit import transpile

basic_shor = transpile(no_qft, basis_gates=["cx", "h", "x", "rz"], optimization_level=0)
basic_shor.count_ops()

OrderedDict([('rz', 2583), ('cx', 2405), ('h', 758), ('x', 73)])

In [21]:
qiskit_shor = transpile(no_qft, optimization_level=3)
qiskit_shor.count_ops()

OrderedDict([('cx', 2373),
             ('t', 1151),
             ('tdg', 1091),
             ('u2', 326),
             ('h', 320),
             ('x', 47),
             ('unitary', 8)])

In [22]:
syn_shor = our_transpile(no_qft)
syn_shor.count_ops()

[11, 10, 12, 9, 13, 0, 2, 1, 4, 3, 6, 5, 8, 7, 14]


OrderedDict([('cx', 3324),
             ('rz', 2628),
             ('h', 1010),
             ('sx', 337),
             ('x', 7),
             ('z', 5),
             ('s', 2),
             ('permutation', 1)])

In [ ]:
check_equivalent(no_qft, syn_shor)

In [24]:
#check_equivalent(no_qft, qiskit_shor)

In [25]:
#check_equivalent(no_qft, basic_shor)